# Xgboost
Xgboost 참고 : https://dacon.io/competitions/official/235689/codeshare/2412?page=1&dtype=recent&ptype=pub

In [52]:
import pandas as pd

In [53]:
train_feature = pd.read_csv('./data/train_features.csv')
test_feature = pd.read_csv('./data/test_features.csv')
train_label = pd.read_csv('./data/train_labels.csv')
sample_submission = pd.read_csv('./data/sample_submission.csv')

In [55]:
sample_submission

,id,0,1,2,3,4,5,6,7,8,...,51,52,53,54,55,56,57,58,59,60
0,3125,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3126,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3127,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3128,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,3129,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,3902,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
778,3903,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
779,3904,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
780,3905,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
X_data = train_feature[['id', 'acc_x', 'acc_y', 'acc_z', 'gy_x', 'gy_y', 'gy_z']]
X_exam = test_feature[['id', 'acc_x', 'acc_y', 'acc_z', 'gy_x', 'gy_y', 'gy_z']]
Y_data = train_label['label']

In [4]:
X_data

,id,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112
1,0,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216
2,0,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629
3,0,1.293095,-0.230366,-0.215210,2.712986,-53.597843,-27.454013
4,0,1.300887,-0.187757,-0.222523,4.286707,-57.906561,-27.961234
...,...,...,...,...,...,...,...
1874995,3124,-0.712530,-0.658357,0.293707,-29.367857,-104.013664,-76.290437
1874996,3124,-0.683037,-0.658466,0.329223,-30.149089,-101.796809,-76.625087
1874997,3124,-0.664730,-0.666625,0.364114,-27.873095,-98.776072,-79.365125
1874998,3124,-0.630534,-0.682565,0.373696,-23.636550,-99.139495,-80.259478


In [5]:
X_exam

,id,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,3125,-0.628100,-0.160155,0.151487,49.665357,88.435961,13.597668
1,3125,-0.462548,0.012462,-0.053726,56.953059,96.185341,16.278458
2,3125,-0.363481,-0.091789,-0.130004,29.557396,93.836453,13.329043
3,3125,-0.351750,-0.239870,-0.193053,23.686172,88.608721,13.449771
4,3125,-0.312934,-0.123762,-0.318621,20.410071,85.327707,13.884912
...,...,...,...,...,...,...,...
469195,3906,0.104191,-0.784979,0.639513,-10.475346,14.095361,-190.358982
469196,3906,0.103297,-0.758954,0.615687,-25.360272,-8.523018,-180.393291
469197,3906,0.128294,-0.749389,0.586184,-27.917723,-23.186245,-162.624160
469198,3906,0.104130,-0.692731,0.573397,-27.847980,-30.407555,-138.761676


In [6]:
Y_data

0       37
1       26
2        3
3       26
4       26
        ..
3120    26
3121    26
3122    15
3123    26
3124     2
Name: label, Length: 3125, dtype: int64

In [7]:
def q1(x):
    return x.quantile(0.25)

def q3(x):
    return x.quantile(0.75)

def IQR(x):
    return q3(x) - q1(x)

def RMS(x):
    return np.sqrt(np.mean(x**2))

def ZCR(x):
    return (((np.array(x)[:-1] * np.array(x)[1:]) < 0).sum())/len(x)

In [8]:
import numpy as np
from numba import jit # 파이썬과 넘파이 코드를 빠르게 실행시켜주는 JIT 컴파일러라고 할 수 있겠다.
from math import factorial, log
from sklearn.neighbors import KDTree
from scipy.signal import periodogram, welch # 주기율

In [9]:
# 비화상
def spectral_entropy(x, sf = 600, method='fft', nperseg=None, normalize=False):
    x = np.array(x)
    # Compute and normalize power spectrum
    if method == 'fft':
        _, psd = periodogram(x, sf)
    elif method == 'welch':
        _, psd = welch(x, sf, nperseg=nperseg)
    psd_norm = np.divide(psd, psd.sum())
    se = -np.multiply(psd_norm, np.log2(psd_norm)).sum()
    if normalize:
        se /= np.log2(psd_norm.size)
    return se

In [10]:
X_train_1 = X_data.groupby('id').agg(['max', 'min', 'mean', 'mad', q1, q3, IQR, RMS, ZCR, spectral_entropy])

C:\ProgramData\Anaconda3\envs\AI_dev_env3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in log2
  if __name__ == '__main__':
C:\ProgramData\Anaconda3\envs\AI_dev_env3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in multiply
  if __name__ == '__main__':


In [11]:
X_test_1 = X_exam.groupby('id').agg(['max', 'min', 'mean', 'mad', q1, q3, IQR, RMS, ZCR, spectral_entropy])

C:\ProgramData\Anaconda3\envs\AI_dev_env3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in log2
  if __name__ == '__main__':
C:\ProgramData\Anaconda3\envs\AI_dev_env3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in multiply
  if __name__ == '__main__':


In [12]:
X_train_1.columns

MultiIndex([('acc_x',              'max'),
            ('acc_x',              'min'),
            ('acc_x',             'mean'),
            ('acc_x',              'mad'),
            ('acc_x',               'q1'),
            ('acc_x',               'q3'),
            ('acc_x',              'IQR'),
            ('acc_x',              'RMS'),
            ('acc_x',              'ZCR'),
            ('acc_x', 'spectral_entropy'),
            ('acc_y',              'max'),
            ('acc_y',              'min'),
            ('acc_y',             'mean'),
            ('acc_y',              'mad'),
            ('acc_y',               'q1'),
            ('acc_y',               'q3'),
            ('acc_y',              'IQR'),
            ('acc_y',              'RMS'),
            ('acc_y',              'ZCR'),
            ('acc_y', 'spectral_entropy'),
            ('acc_z',              'max'),
            ('acc_z',              'min'),
            ('acc_z',             'mean'),
           

In [13]:
X_train_1

acc_x                                                              \
           max       min      mean       mad        q1        q3       IQR   
id                                                                           
0     1.344268  0.591940  0.931329  0.168768  0.747302  1.080077  0.332775   
1     1.234020 -2.156208 -0.766580  0.394125 -1.057583 -0.383457  0.674126   
2     1.219836 -1.142847  0.039836  0.650191 -0.688672  0.736468  1.425141   
3    -0.622250 -1.417751 -0.887702  0.105425 -0.982072 -0.784699  0.197374   
4     0.599720 -2.429109 -0.659018  0.426308 -0.960309 -0.347234  0.613075   
...        ...       ...       ...       ...       ...       ...       ...   
3120  0.390798 -1.624711 -0.300454  0.337574 -0.588712 -0.086355  0.502357   
3121 -0.446650 -1.575455 -0.974298  0.113140 -1.021473 -0.909872  0.111601   
3122  0.744666 -2.578974 -1.114246  0.591276 -1.697727 -0.569290  1.128437   
3123  0.915846 -0.929133 -0.111333  0.356627 -0.458019  0.223488  0.681508   
3124  0.538809 -1.013813 -0.434048  0.470846 -0.935463  0.089389  1.024852   

                                           ...        gy_z              \
           RMS       ZCR spectral_entropy  ...         max         min   
id                                         ...                           
0     0.950777  0.000000         2.918811  ...   55.953827  -79.930029   
1     0.912570  0.011667         4.389594  ...  340.170199 -270.980823   
2     0.712493  0.035000         2.432518  ...   55.642836  -44.192071   
3     0.897285  0.000000         5.388780  ...   56.456908  -85.600536   
4     0.824069  0.035000         3.137033  ...  221.015193 -270.581913   
...        ...       ...              ...  ...         ...         ...   
3120  0.502545  0.025000         2.614809  ...  121.958427  -79.392292   
3121  0.988987  0.000000         4.547195  ...   57.349878  -39.777626   
3122  1.307032  0.016667         2.929578  ...  453.943910 -247.908573   
3123  0.446465  0.040000         3.497426  ...  310.558507 -206.580638   
3124  0.678630  0.016667         0.646104  ...   80.302840 -100.845205   

                                                                            \
          mean         mad          q1          q3         IQR         RMS   
id                                                                           
0     1.182107   19.159499   -9.930266   19.479614   29.409880   25.281765   
1     1.393294   49.925202  -30.514605   37.521852   68.036458   75.495221   
2     3.053291   10.464255   -4.718664   10.915188   15.633852   14.239923   
3    -5.869898   17.702520  -19.096905    7.843401   26.940305   24.345668   
4     4.453382   24.457885   -3.318290   12.739489   16.057779   46.324412   
...        ...         ...         ...         ...         ...         ...   
3120 -0.054026   14.882663   -5.959514    2.601295    8.560809   24.893107   
3121 -2.792238    8.113331   -6.335722    1.838386    8.174108   13.077376   
3122 -1.722830  115.714812 -119.508096  110.101101  229.609196  131.817892   
3123 -5.930252   50.511538  -46.728375   23.479180   70.207555   71.430350   
3124 -3.246825   42.944335  -50.133093   42.920979   93.054072   49.436692   

                                 
           ZCR spectral_entropy  
id                               
0     0.081667         3.164716  
1     0.048333         4.538745  
2     0.145000         5.866290  
3     0.086667         4.893066  
4     0.078333         5.546258  
...        ...              ...  
3120  0.060000         4.687651  
3121  0.086667         4.990858  
3122  0.031667         2.816127  
3123  0.053333         4.253495  
3124  0.013333         0.587293  

[3125 rows x 60 columns]

In [14]:
columns_rename = []
for i, j in X_train_1.columns: # 튜플입력
    rename= str(i) + str('_') + str(j)
    columns_rename.append(rename)
X_train_1.columns = columns_rename
X_test_1.columns = columns_rename

In [15]:
X_train_1

,acc_x_max,acc_x_min,acc_x_mean,acc_x_mad,acc_x_q1,acc_x_q3,acc_x_IQR,acc_x_RMS,acc_x_ZCR,acc_x_spectral_entropy,...,gy_z_max,gy_z_min,gy_z_mean,gy_z_mad,gy_z_q1,gy_z_q3,gy_z_IQR,gy_z_RMS,gy_z_ZCR,gy_z_spectral_entropy
id,,,,,,,,,,,,,,,,,,,,,
0,1.344268,0.591940,0.931329,0.168768,0.747302,1.080077,0.332775,0.950777,0.000000,2.918811,...,55.953827,-79.930029,1.182107,19.159499,-9.930266,19.479614,29.409880,25.281765,0.081667,3.164716
1,1.234020,-2.156208,-0.766580,0.394125,-1.057583,-0.383457,0.674126,0.912570,0.011667,4.389594,...,340.170199,-270.980823,1.393294,49.925202,-30.514605,37.521852,68.036458,75.495221,0.048333,4.538745
2,1.219836,-1.142847,0.039836,0.650191,-0.688672,0.736468,1.425141,0.712493,0.035000,2.432518,...,55.642836,-44.192071,3.053291,10.464255,-4.718664,10.915188,15.633852,14.239923,0.145000,5.866290
3,-0.622250,-1.417751,-0.887702,0.105425,-0.982072,-0.784699,0.197374,0.897285,0.000000,5.388780,...,56.456908,-85.600536,-5.869898,17.702520,-19.096905,7.843401,26.940305,24.345668,0.086667,4.893066
4,0.599720,-2.429109,-0.659018,0.426308,-0.960309,-0.347234,0.613075,0.824069,0.035000,3.137033,...,221.015193,-270.581913,4.453382,24.457885,-3.318290,12.739489,16.057779,46.324412,0.078333,5.546258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3120,0.390798,-1.624711,-0.300454,0.337574,-0.588712,-0.086355,0.502357,0.502545,0.025000,2.614809,...,121.958427,-79.392292,-0.054026,14.882663,-5.959514,2.601295,8.560809,24.893107,0.060000,4.687651
3121,-0.446650,-1.575455,-0.974298,0.113140,-1.021473,-0.909872,0.111601,0.988987,0.000000,4.547195,...,57.349878,-39.777626,-2.792238,8.113331,-6.335722,1.838386,8.174108,13.077376,0.086667,4.990858
3122,0.744666,-2.578974,-1.114246,0.591276,-1.697727,-0.569290,1.128437,1.307032,0.016667,2.929578,...,453.943910,-247.908573,-1.722830,115.714812,-119.508096,110.101101,229.609196,131.817892,0.031667,2.816127


In [16]:
X_train_1.isnull().sum()

acc_x_max                  0
acc_x_min                  0
acc_x_mean                 0
acc_x_mad                  0
acc_x_q1                   0
acc_x_q3                   0
acc_x_IQR                  0
acc_x_RMS                  0
acc_x_ZCR                  0
acc_x_spectral_entropy     9
acc_y_max                  0
acc_y_min                  0
acc_y_mean                 0
acc_y_mad                  0
acc_y_q1                   0
acc_y_q3                   0
acc_y_IQR                  0
acc_y_RMS                  0
acc_y_ZCR                  0
acc_y_spectral_entropy     9
acc_z_max                  0
acc_z_min                  0
acc_z_mean                 0
acc_z_mad                  0
acc_z_q1                   0
acc_z_q3                   0
acc_z_IQR                  0
acc_z_RMS                  0
acc_z_ZCR                  0
acc_z_spectral_entropy    17
gy_x_max                   0
gy_x_min                   0
gy_x_mean                  0
gy_x_mad                   0
gy_x_q1       

In [205]:
# X_train_1.fillna(0)
# X_test_1.fillna(0)
# X_train_1.shape

(3125, 60)

In [17]:
X_train_abs = np.abs(X_data)
X_train_abs

,id,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,0,1.206087,0.179371,0.148447,0.591608,30.549010,31.676112
1,0,1.287696,0.198974,0.182444,0.303100,39.139103,24.927216
2,0,1.304609,0.195114,0.253382,3.617278,44.122565,25.019629
3,0,1.293095,0.230366,0.215210,2.712986,53.597843,27.454013
4,0,1.300887,0.187757,0.222523,4.286707,57.906561,27.961234
...,...,...,...,...,...,...,...
1874995,3124,0.712530,0.658357,0.293707,29.367857,104.013664,76.290437
1874996,3124,0.683037,0.658466,0.329223,30.149089,101.796809,76.625087
1874997,3124,0.664730,0.666625,0.364114,27.873095,98.776072,79.365125
1874998,3124,0.630534,0.682565,0.373696,23.636550,99.139495,80.259478


In [18]:
X_train_change = X_train_abs.groupby('id').agg(['mean']) # 여러개의 함수를 여러 열에 적용
X_train_change

,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
,mean,mean,mean,mean,mean,mean
id,,,,,,
0,0.931329,0.233546,0.370422,10.539597,18.133371,19.210501
1,0.778600,0.359997,0.404201,56.442665,61.585563,49.884768
2,0.652609,0.118084,0.626012,19.273546,101.395224,10.875752
3,0.887702,0.166819,0.253105,32.823604,26.202033,18.102910
4,0.721407,0.500824,0.342849,46.996688,28.739933,23.387152
...,...,...,...,...,...,...
3120,0.343886,0.670055,0.449323,20.283010,20.555379,14.903192
3121,0.974298,0.234871,0.094520,22.722197,10.567139,8.413857


In [19]:
X_test_abs = np.abs(X_exam)
X_test_change = X_test_abs.groupby('id').agg(['mean'])

In [20]:
columns_rename_2 = []
for i, j in X_train_change.columns:
    rename = str(i) + str('_') + str(j)
    columns_rename_2.append(rename)
X_train_change.columns = columns_rename_2
X_test_change.columns = columns_rename_2

In [21]:
X_train_1

,acc_x_max,acc_x_min,acc_x_mean,acc_x_mad,acc_x_q1,acc_x_q3,acc_x_IQR,acc_x_RMS,acc_x_ZCR,acc_x_spectral_entropy,...,gy_z_max,gy_z_min,gy_z_mean,gy_z_mad,gy_z_q1,gy_z_q3,gy_z_IQR,gy_z_RMS,gy_z_ZCR,gy_z_spectral_entropy
id,,,,,,,,,,,,,,,,,,,,,
0,1.344268,0.591940,0.931329,0.168768,0.747302,1.080077,0.332775,0.950777,0.000000,2.918811,...,55.953827,-79.930029,1.182107,19.159499,-9.930266,19.479614,29.409880,25.281765,0.081667,3.164716
1,1.234020,-2.156208,-0.766580,0.394125,-1.057583,-0.383457,0.674126,0.912570,0.011667,4.389594,...,340.170199,-270.980823,1.393294,49.925202,-30.514605,37.521852,68.036458,75.495221,0.048333,4.538745
2,1.219836,-1.142847,0.039836,0.650191,-0.688672,0.736468,1.425141,0.712493,0.035000,2.432518,...,55.642836,-44.192071,3.053291,10.464255,-4.718664,10.915188,15.633852,14.239923,0.145000,5.866290
3,-0.622250,-1.417751,-0.887702,0.105425,-0.982072,-0.784699,0.197374,0.897285,0.000000,5.388780,...,56.456908,-85.600536,-5.869898,17.702520,-19.096905,7.843401,26.940305,24.345668,0.086667,4.893066
4,0.599720,-2.429109,-0.659018,0.426308,-0.960309,-0.347234,0.613075,0.824069,0.035000,3.137033,...,221.015193,-270.581913,4.453382,24.457885,-3.318290,12.739489,16.057779,46.324412,0.078333,5.546258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3120,0.390798,-1.624711,-0.300454,0.337574,-0.588712,-0.086355,0.502357,0.502545,0.025000,2.614809,...,121.958427,-79.392292,-0.054026,14.882663,-5.959514,2.601295,8.560809,24.893107,0.060000,4.687651
3121,-0.446650,-1.575455,-0.974298,0.113140,-1.021473,-0.909872,0.111601,0.988987,0.000000,4.547195,...,57.349878,-39.777626,-2.792238,8.113331,-6.335722,1.838386,8.174108,13.077376,0.086667,4.990858
3122,0.744666,-2.578974,-1.114246,0.591276,-1.697727,-0.569290,1.128437,1.307032,0.016667,2.929578,...,453.943910,-247.908573,-1.722830,115.714812,-119.508096,110.101101,229.609196,131.817892,0.031667,2.816127


In [22]:
X_train_change['SMAacc'] = X_train_change['acc_x_mean']+X_train_change['acc_y_mean']+X_train_change['acc_z_mean']
X_train_change['SMAgy'] = X_train_change['gy_x_mean']+X_train_change['gy_y_mean']+X_train_change['gy_z_mean']
X_test_change['SMAacc'] = X_test_change['acc_x_mean']+X_test_change['acc_y_mean']+X_test_change['acc_z_mean']
X_test_change['SMAgy'] = X_test_change['gy_x_mean']+X_test_change['gy_y_mean']+X_test_change['gy_z_mean']

In [23]:
X_train_change

,acc_x_mean,acc_y_mean,acc_z_mean,gy_x_mean,gy_y_mean,gy_z_mean,SMAacc,SMAgy
id,,,,,,,,
0,0.931329,0.233546,0.370422,10.539597,18.133371,19.210501,1.535296,47.883469
1,0.778600,0.359997,0.404201,56.442665,61.585563,49.884768,1.542798,167.912997
2,0.652609,0.118084,0.626012,19.273546,101.395224,10.875752,1.396705,131.544522
3,0.887702,0.166819,0.253105,32.823604,26.202033,18.102910,1.307625,77.128547
4,0.721407,0.500824,0.342849,46.996688,28.739933,23.387152,1.565080,99.123773
...,...,...,...,...,...,...,...,...
3120,0.343886,0.670055,0.449323,20.283010,20.555379,14.903192,1.463265,55.741581
3121,0.974298,0.234871,0.094520,22.722197,10.567139,8.413857,1.303689,41.703193
3122,1.119284,0.390649,0.269586,117.184446,219.362514,115.751137,1.779518,452.298097


In [24]:
X_train_1[['SMAacc', 'SMAgy']] = X_train_change[['SMAacc', 'SMAgy']]
X_test_1[['SMAacc', 'SMAgy']] = X_test_change[['SMAacc', 'SMAgy']]

In [25]:
X_train_1

,acc_x_max,acc_x_min,acc_x_mean,acc_x_mad,acc_x_q1,acc_x_q3,acc_x_IQR,acc_x_RMS,acc_x_ZCR,acc_x_spectral_entropy,...,gy_z_mean,gy_z_mad,gy_z_q1,gy_z_q3,gy_z_IQR,gy_z_RMS,gy_z_ZCR,gy_z_spectral_entropy,SMAacc,SMAgy
id,,,,,,,,,,,,,,,,,,,,,
0,1.344268,0.591940,0.931329,0.168768,0.747302,1.080077,0.332775,0.950777,0.000000,2.918811,...,1.182107,19.159499,-9.930266,19.479614,29.409880,25.281765,0.081667,3.164716,1.535296,47.883469
1,1.234020,-2.156208,-0.766580,0.394125,-1.057583,-0.383457,0.674126,0.912570,0.011667,4.389594,...,1.393294,49.925202,-30.514605,37.521852,68.036458,75.495221,0.048333,4.538745,1.542798,167.912997
2,1.219836,-1.142847,0.039836,0.650191,-0.688672,0.736468,1.425141,0.712493,0.035000,2.432518,...,3.053291,10.464255,-4.718664,10.915188,15.633852,14.239923,0.145000,5.866290,1.396705,131.544522
3,-0.622250,-1.417751,-0.887702,0.105425,-0.982072,-0.784699,0.197374,0.897285,0.000000,5.388780,...,-5.869898,17.702520,-19.096905,7.843401,26.940305,24.345668,0.086667,4.893066,1.307625,77.128547
4,0.599720,-2.429109,-0.659018,0.426308,-0.960309,-0.347234,0.613075,0.824069,0.035000,3.137033,...,4.453382,24.457885,-3.318290,12.739489,16.057779,46.324412,0.078333,5.546258,1.565080,99.123773
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3120,0.390798,-1.624711,-0.300454,0.337574,-0.588712,-0.086355,0.502357,0.502545,0.025000,2.614809,...,-0.054026,14.882663,-5.959514,2.601295,8.560809,24.893107,0.060000,4.687651,1.463265,55.741581
3121,-0.446650,-1.575455,-0.974298,0.113140,-1.021473,-0.909872,0.111601,0.988987,0.000000,4.547195,...,-2.792238,8.113331,-6.335722,1.838386,8.174108,13.077376,0.086667,4.990858,1.303689,41.703193
3122,0.744666,-2.578974,-1.114246,0.591276,-1.697727,-0.569290,1.128437,1.307032,0.016667,2.929578,...,-1.722830,115.714812,-119.508096,110.101101,229.609196,131.817892,0.031667,2.816127,1.779518,452.298097


In [26]:
from sklearn.metrics import accuracy_score

In [27]:
print(X_train_1.shape)
print(Y_data.shape)

(3125, 62)
(3125,)


In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_train_1, Y_data, 
                                                    test_size=0.2, random_state=26, 
                                                    stratify=Y_data) # 값이 치우쳐지는것을 방지

In [48]:
from xgboost import XGBClassifier
xgb_clf = XGBClassifier(n_estimators=1000, learning_rate = 0.1, max_depth=3)

In [49]:
%%time

evals = [(X_test, Y_test)] # A list of (X, y) tuple pairs

xgb_clf.fit(X_train, Y_train, early_stopping_rounds=400, eval_set=evals, eval_metric='mlogloss', verbose=True)
xgb_pred = xgb_clf.predict(X_test)
xgb_pred_proba = xgb_clf.predict_proba(X_test)[:, 1]

# mlogloss: multiple class (cf. binary class)
# verbose=True: 학습 로그 찍기
# predict_proba: 예측 확률(cross entropy 때문에 그렇다)

[0]	validation_0-mlogloss:2.84187
Will train until validation_0-mlogloss hasn't improved in 400 rounds.
[1]	validation_0-mlogloss:2.55916
[2]	validation_0-mlogloss:2.35005
[3]	validation_0-mlogloss:2.17898
[4]	validation_0-mlogloss:2.04057
[5]	validation_0-mlogloss:1.91999
[6]	validation_0-mlogloss:1.81654
[7]	validation_0-mlogloss:1.72753
[8]	validation_0-mlogloss:1.64659
[9]	validation_0-mlogloss:1.57545
[10]	validation_0-mlogloss:1.51432
[11]	validation_0-mlogloss:1.45695
[12]	validation_0-mlogloss:1.40708
[13]	validation_0-mlogloss:1.36251
[14]	validation_0-mlogloss:1.3198
[15]	validation_0-mlogloss:1.28048
[16]	validation_0-mlogloss:1.24593
[17]	validation_0-mlogloss:1.21307
[18]	validation_0-mlogloss:1.18287
[19]	validation_0-mlogloss:1.15696
[20]	validation_0-mlogloss:1.13005
[21]	validation_0-mlogloss:1.1058
[22]	validation_0-mlogloss:1.08349
[23]	validation_0-mlogloss:1.06335
[24]	validation_0-mlogloss:1.04464
[25]	validation_0-mlogloss:1.02577
[26]	validation_0-mlogloss:1.009

[225]	validation_0-mlogloss:0.767777
[226]	validation_0-mlogloss:0.76824
[227]	validation_0-mlogloss:0.768186
[228]	validation_0-mlogloss:0.768624
[229]	validation_0-mlogloss:0.768842
[230]	validation_0-mlogloss:0.768849
[231]	validation_0-mlogloss:0.769073
[232]	validation_0-mlogloss:0.769116
[233]	validation_0-mlogloss:0.768983
[234]	validation_0-mlogloss:0.769217
[235]	validation_0-mlogloss:0.769373
[236]	validation_0-mlogloss:0.769431
[237]	validation_0-mlogloss:0.769388
[238]	validation_0-mlogloss:0.769321
[239]	validation_0-mlogloss:0.769573
[240]	validation_0-mlogloss:0.769812
[241]	validation_0-mlogloss:0.769787
[242]	validation_0-mlogloss:0.769744
[243]	validation_0-mlogloss:0.769766
[244]	validation_0-mlogloss:0.769885
[245]	validation_0-mlogloss:0.770221
[246]	validation_0-mlogloss:0.770369
[247]	validation_0-mlogloss:0.770156
[248]	validation_0-mlogloss:0.770317
[249]	validation_0-mlogloss:0.770094
[250]	validation_0-mlogloss:0.770206
[251]	validation_0-mlogloss:0.770214
[2

[448]	validation_0-mlogloss:0.773098
[449]	validation_0-mlogloss:0.773108
[450]	validation_0-mlogloss:0.773034
[451]	validation_0-mlogloss:0.7732
[452]	validation_0-mlogloss:0.772876
[453]	validation_0-mlogloss:0.772804
[454]	validation_0-mlogloss:0.772742
[455]	validation_0-mlogloss:0.772842
[456]	validation_0-mlogloss:0.772857
[457]	validation_0-mlogloss:0.772929
[458]	validation_0-mlogloss:0.772851
[459]	validation_0-mlogloss:0.773033
[460]	validation_0-mlogloss:0.772728
[461]	validation_0-mlogloss:0.772847
[462]	validation_0-mlogloss:0.772867
[463]	validation_0-mlogloss:0.772846
[464]	validation_0-mlogloss:0.772803
[465]	validation_0-mlogloss:0.772851
[466]	validation_0-mlogloss:0.772683
[467]	validation_0-mlogloss:0.772788
[468]	validation_0-mlogloss:0.772678
[469]	validation_0-mlogloss:0.772629
[470]	validation_0-mlogloss:0.772515
[471]	validation_0-mlogloss:0.772543
[472]	validation_0-mlogloss:0.772302
[473]	validation_0-mlogloss:0.772215
[474]	validation_0-mlogloss:0.772262
[47

In [50]:
xgb_accuracy = accuracy_score(Y_test, xgb_pred)
print(f'GBM 정확도 : {xgb_accuracy:.4f}')

GBM 정확도 : 0.7984


In [225]:
xgb_pred.shape

(625,)

In [226]:
Y_test.shape

(625,)

In [222]:
from xgboost import XGBClassifier

xgb = XGBClassifier()
xgb.fit(X_train_1, y_train)
xgbpred = xgb.predict_proba(X_test_1)

In [51]:
sample_submission.iloc[:, 1:] = xgb_pred
sample_submission.head()

ValueError: could not broadcast input array from shape (625) into shape (782,61)